### 1. Install and Import Dependencies

In [3]:
!pip install mediapipe opencv-python
!pip install ultralytics
!pip install pybboxes

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip


In [8]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import time
from ultralytics import YOLO
import pybboxes as pbx
from pybboxes import convert_bbox

### 2. YOLO

In [5]:
weights_loc = "/Users/taylor/runs/detect/train14/weights/best.pt"
yolo_model = YOLO(weights_loc)

### 3. Hand poses

In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [30]:
#integrate yolo and hand detection
cap = cv2.VideoCapture(0)
hands_list = []

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()

        image_height, image_width, _ = frame.shape

        #image stuff
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_height, image_width, _ = image.shape
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        yolo_results = yolo_model.predict(frame)
        print(yolo_results[0])

        pose_results = hands.process(image)

        # Rendering results
        if pose_results.multi_hand_landmarks:
            for handLandmarks in pose_results.multi_hand_landmarks:
                hand_dict = {}
                for point in mp_hands.HandLandmark:
                    normalizedLandmark = handLandmarks.landmark[point]
                    pixelCoordinatesLandmark = mp_drawing._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, image_width, image_height)
                    hand_dict[point.name] = pixelCoordinatesLandmark
                hands_list.append(hand_dict)

            for num, hand in enumerate(pose_results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                )
            
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('s'):
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            filename = f"HandTracking_{timestamp}.png"
            cv2.imwrite("saved_imgs/" + filename, image)
            print(f"Image saved: {filename}")

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 117.8ms
Speed: 2.5ms preprocess, 117.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'myleft', 1: 'myright', 2: 'yourleft', 3: 'yourright'}
obb: None
orig_img: array([[[2, 3, 1],
        [2, 3, 1],
        [2, 3, 1],
        ...,
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0]],

       [[2, 3, 1],
        [2, 3, 1],
        [2, 3, 1],
        ...,
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0]],

       [[2, 3, 1],
        [2, 3, 1],
        [2, 3, 1],
        ...,
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0]],

       ...,

       [[2, 3, 1],
        [2, 3, 1],
        [2, 3, 1],
        ...,
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0]],

       [[2, 3, 1],
        [2, 3, 1],
        [2, 3, 1],
        ...,
        [1, 2, 0],
        [1, 2, 0],
       

In [ ]:
for val in hands_list:
    print(val)

{'WRIST': None, 'THUMB_CMC': (404, 1036), 'THUMB_MCP': (345, 865), 'THUMB_IP': (309, 716), 'THUMB_TIP': (243, 611), 'INDEX_FINGER_MCP': (549, 712), 'INDEX_FINGER_PIP': (531, 530), 'INDEX_FINGER_DIP': (525, 413), 'INDEX_FINGER_TIP': (527, 317), 'MIDDLE_FINGER_MCP': (677, 729), 'MIDDLE_FINGER_PIP': (724, 532), 'MIDDLE_FINGER_DIP': (755, 417), 'MIDDLE_FINGER_TIP': (783, 321), 'RING_FINGER_MCP': (784, 795), 'RING_FINGER_PIP': (865, 616), 'RING_FINGER_DIP': (922, 506), 'RING_FINGER_TIP': (975, 407), 'PINKY_MCP': (865, 900), 'PINKY_PIP': (960, 803), 'PINKY_DIP': (1018, 742), 'PINKY_TIP': (1073, 681)}
{'WRIST': (922, 1039), 'THUMB_CMC': (767, 916), 'THUMB_MCP': (693, 730), 'THUMB_IP': (674, 577), 'THUMB_TIP': (614, 468), 'INDEX_FINGER_MCP': (859, 553), 'INDEX_FINGER_PIP': (836, 349), 'INDEX_FINGER_DIP': (831, 220), 'INDEX_FINGER_TIP': (834, 109), 'MIDDLE_FINGER_MCP': (990, 563), 'MIDDLE_FINGER_PIP': (1033, 344), 'MIDDLE_FINGER_DIP': (1066, 206), 'MIDDLE_FINGER_TIP': (1099, 89), 'RING_FINGER_M

: 

: 